# Aufgabe 3: Gaußfilter
Anstatt wie in der vorigen Aufgabe an einer ausgewählten Frequenz hart abzuschneiden, kann mittels eines Faktors
\begin{align}
 h(u,v) = \mathrm{e}^{-\frac{D^2(u,v)}{2\sigma^2}},\qquad h \in [0;1]
\end{align}
auch ein weicher Übergang erzeugt werden.

Verwenden Sie eine Gauß-Glocke für die Hoch- und Tiefpassfilterung und berechnen Sie den Faktor $h$ in Abhängigkeit von der Entfernung $D(u,v)$ zum Ursprung!
Führen Sie die Transformationen ansonsten wie in Teilaufgabe 2 durch! Die Varianz $\sigma$ der Gauß-Glocke ist mit dem dort gegebenen Radius gleichzusetzen.
Was ändert sich in den Ausgabebildern im Vergleich zu Teilaufgabe 2?

## 0. Pfade, Pakete etc.

In [ ]:
import glob
import imageio
import numpy as np
import math

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
image_filter = 'Bilder/*.jpg'

## 1. Definition des Filters

Übernehmen Sie hier zu Vergleichszwecken Ihre Definition des idealen Hoch- und Tiefpass-Filters aus der vorigen Aufgabe:

In [ ]:
D0 = 20
lowpass_filter = lambda D: int(D <= D0)
highpass_filter = lambda D: int(D > D0)

Definieren Sie den Gauß-Filter als Funktion des Radius $D$ und der Konstante $\sigma$:

In [ ]:
sigma = 20
gauss_lowpass = lambda D: np.exp(-D**2/(2*sigma**2))
gauss_highpass = lambda D: 1-np.exp(-D**2/(2*sigma**2))

## 2. Laden und Normalisieren des Bildes

In [ ]:
image_path = np.random.choice(glob.glob(image_filter))
image = imageio.imread(image_path)

In [ ]:
image = image.astype(np.float32)
image -= image.min()
image /= image.max()

## 3. Berechnung der Fouriertransformation
Setzen Sie hier ihre Lösung aus der vorigen Aufgabe ein:

In [ ]:
image_transformed = np.fft.fftshift(np.fft.fft2(image))

## 4. Filterung

Setzen Sie hier ihre Lösung `ex3_filter_spectrum` aus der vorigen Aufgabe ein:

In [ ]:
def ex3_filter_spectrum(spectrum, radial_filter):
    spectrum_copy = spectrum.copy()
    y, x = image.shape

    for v in range(y):
        for u in range(x):
            D = np.sqrt((v-y/2)**2 + (u-x/2)**2)
            spectrum_copy[v,u] *= radial_filter(D)

    return spectrum_copy

Das transformierte Bild (Spektrum) wird nun gefiltert:

In [ ]:
image_transformed_lowpass = ex3_filter_spectrum(image_transformed, lowpass_filter)
image_transformed_highpass = ex3_filter_spectrum(image_transformed, highpass_filter)
image_transformed_gauss_lowpass = ex3_filter_spectrum(image_transformed, gauss_lowpass)
image_transformed_gauss_highpass = ex3_filter_spectrum(image_transformed, gauss_highpass)

## 5. Inverse Filterung
Das veränderte Spektrum soll nun in den Ortsbereich zurücktransformiert werden. Verwenden Sie dazu die entsprechenden Funktionen des Paketes `numpy.fft`.

In [ ]:
image_filter_lowpass = np.fft.ifft2(np.fft.ifftshift(image_transformed_lowpass)).real
image_filter_highpass = np.fft.ifft2(np.fft.ifftshift(image_transformed_highpass)).real
image_filter_gauss_lowpass = np.fft.ifft2(np.fft.ifftshift(image_transformed_gauss_lowpass)).real
image_filter_gauss_highpass = np.fft.ifft2(np.fft.ifftshift(image_transformed_gauss_highpass)).real

Vergleichen Sie nun die Ergebnisses des idealen Hoch- und Tiefpassfilters mit den Gauß-Filtern:

In [ ]:
plt.figure(figsize=(8, 12))
plt.subplot(3,2,1, title='Original Image')
plt.imshow(image, cmap='gray', vmin=0, vmax=1)
plt.subplot(3,2,3, title='Lowpass')
plt.ylabel('Ideal')
plt.imshow(image_filter_lowpass, cmap='gray', vmin=0, vmax=1)
plt.subplot(3,2,4, title='Highpass')
plt.imshow(image_filter_highpass, cmap='gray')
plt.subplot(3,2,5)
plt.ylabel('Gauss')
plt.imshow(image_filter_gauss_lowpass, cmap='gray', vmin=0, vmax=1)
plt.subplot(3,2,6)
plt.imshow(image_filter_gauss_highpass, cmap='gray')
plt.show()

### Welche Unterschiede lassen sich im Vergleich zu Aufgabe 2 feststellen?

Zwischen den niedrigen Frequenzen, die wir durchlassen und den hohen, die wir raus schneiden ist im Ideal Filter eine scharfe Kante. Diese verursacht bei der Rück-Transformation hohe wellenartige Frequenzen.
Wo hingegen der Gauss-Filtern nach der Rück-Transformation immernoch wie ein Gauss-Filter aussieht.

In [ ]:
y, x = image.shape
gauss_mask_transformed = image.copy()
ideal_mask_transformed = image.copy()

for v in range(y):
    for u in range(x):
        D = np.sqrt((v-y/2)**2 + (u-x/2)**2)
        ideal_mask_transformed[v,u] = lowpass_filter(D)
        gauss_mask_transformed[v,u] = gauss_lowpass(D)


plt.figure(figsize=(10, 10))
plt.subplot(2,2,1, title='Frequenzbereich')
plt.ylabel('Ideal')
plt.imshow(ideal_mask_transformed)
plt.subplot(2,2,2, title='Ortsbereich')
plt.imshow(np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(ideal_mask_transformed))).real)
plt.subplot(2,2,3)
plt.ylabel('Gauss')
plt.imshow(gauss_mask_transformed)
plt.subplot(2,2,4)
plt.imshow(np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(gauss_mask_transformed))).real)
plt.show()